In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 함수(필수)

## 네이버 쇼핑 크롤링
- 셀레니움 이용(동적크롤링)
- 검색어 전달 후 표시되는 제품은 일부 제품
    - 스크롤바를 완전히 내려야 전체제품이 나타남
    - 자동화 방식을 이용해 전체제품 표출 후 크롤링 

In [3]:
driver = webdriver.Chrome('/driver/chromedriver.exe')

C:\Users\82107\AppData\Local\Temp\ipykernel_36472\2622547941.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/driver/chromedriver.exe')


In [4]:
# 웹사이트
driver.get("https://www.naver.com")
driver.implicitly_wait(10) # 10초 대기

In [5]:
# 쇼핑메뉴 클릭
driver.find_element(By.CSS_SELECTOR,'a.nav.shop').click()
time.sleep(2)

In [51]:
#__next > div > div.pcHeader_header__tXOY4 > div > div > div._gnb_header_area_150KE > div > div._gnbLogo_gnb_logo_3eIAf > div > div._gnbSearch_gnb_search_3O1L2 > form > div._gnbSearch_inner_2Zksb > div._searchInput_search_input_QXUFf > input

In [6]:
# 검색창 선택 후 클릭
search = driver.find_element(By.CSS_SELECTOR,'#__next > div > div.pcHeader_header__tXOY4 > div > div > div._gnb_header_area_150KE > div > div._gnbLogo_gnb_logo_3eIAf > div > div._gnbSearch_gnb_search_3O1L2 > form > div._gnbSearch_inner_2Zksb > div > input')
search.click()

In [7]:
# 검색어 입력
search.send_keys('아이폰13')

In [8]:
# 엔터키 전송 
search.send_keys(Keys.ENTER)

#### 스크롤의 위치에 따라 표출 제품 갯수가 다르므로
     - 스크롤을 최대한 내려서 검색된 모든 제품을 표출 후 
         -현재 스크롤의 위치 얻어오기(자바스크립트 명령어 사용)
             - return window.scrolly
     - 크롤링 진행

In [9]:
# 스크롤 실행 전 높이 
before_h = driver.execute_script('return window.scrollY')
before_h

0

In [10]:
# 무한 스크롤
# body태그 객체를 찾고 키보드의 end키를 전송
while True :
    # 현재 화면의 맨 아래로 스크롤을 내리기
    driver.find_element(By.CSS_SELECTOR,'body').send_keys(Keys.END)
    # 스크롤 후 페이지 로딩시간 대기
    time.sleep(1)
    
    # 스크롤 후 현재 위치
    after_h = driver.execute_script('return window.scrollY')
    print(after_h)
    if after_h == before_h :
        break
    before_h = after_h

11270.400390625
11270.400390625


In [ ]:
# interval = 2
# prev_height = browser.execute_script("return document.body.scrollHeight")

# while True:
#     #화면 가장 아래로 스크롤 내리기
#     browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
#     time.sleep(interval)

#     curr_height = browser.execute_script("return document.body.scrollHeight")
#     if curr_height == prev_height:
#         break
    
#     prev_height = curr_height

In [ ]:
# 상품 정보  div.basicList_info_area__TWvzp

In [11]:
# 상품정보 추출 클래스명 : .basicList_info_area__TWvzp
items = driver.find_elements(By.CSS_SELECTOR,".basicList_info_area__TWvzp")

In [ ]:
# .basicList_title__VfX3c

In [ ]:
#.price_price__LEGN7

In [12]:
#.basicList_title__VfX3c
for item in items : 
    name = item.find_element(By.CSS_SELECTOR,'.basicList_title__VfX3c').text
    try :
        price = item.find_element(By.CSS_SELECTOR,".price_price__LEGN7").text
    except : 
        price = '판매중단'
    link = item.find_element(By.CSS_SELECTOR,'.basicList_title__VfX3c > a').get_attribute('href')
    print(name,price,link)

아이폰 13 mini 자급제 128GB 미드나이트 Apple 874,000원 https://adcr.naver.com/adcr?x=CxO4zfkTRJPWIb7V+Zvu0f///w==kmTj2DS+5qaxkKGFadr9J1XypNBbKd5+BnBVf1XKU31GzVH+fhAylN6VsTFsAXEPuvNTwelIi4g+MQRMMeZGdSf5KgN15QtntEPjNOXoIjjw7xIaLdncKW7gNGqjRot2ilV9pXmyn3NlwaFBr+LAwHDzAWGQblgPHWBgtDYwCzRY+xqjR16eyyFKRSIj5hl1TFhFgini9ZEH/yC88DWXNgwAenRxhpDm3W39Fs+5jOnf7BAryJoq0K/CZLXwUifutjgwlWCL/HQK5ZfAOZM7a8pcrIxe+ZbpYpUn3HyTEyJ7R8RcaZ6yDYS8zMp+nvsIhtkLQJ7Oje3mTscc3+CHnMZXQLfWYDrRRE8iq2iR2h45wYhVOx9OsgnGi+MllSMxvxnV/X4B2KRftB/kVZVnAKmYH87d6KKUiD5AxaQXzQQXuqjpRs3pMh0SXifxRqNcxTuKdfV0wVp5soXTdde9s/tjdfCaVqDbp9ZpolSmXJXJvtuQiKLQ29iJMFT8ASQ4P0vbmCm3tYZHOrYdT1AsjiwPctVliIbwGSZ9sZJb9nxH94BuS0UpftjAaVl/h/BkVgBraDDz383g60d/0hNArwzWWPxM0ru2kABuVJ2WyktiuDpO3ZmPf5BIVqcRxUhrp3qeQcU1wZ1yKY1mH9UxPP66/7FWNtTPp5LhDtolxAUVPjWKqE2MH2yDMR2WrSjl7Gbfrv4CkbPpbn3/G9poPExoEs0+Jz/lQ4qt0AH6Ti/N0ZNGkcpapT0TENt5cYk+p9b+v3rqESP88DUaAMztlFr/f6mCBATz4wIu+9BOtOjaIrvxGJkKgKxR5+D1R4vS6CqC20V9jo2dfeV4lt9Mv7NQXD4tn6vQP2lWOCog97DButIk1jBgNlX+Ia6KF7RmbhJ

Apple 아이폰 13 프로 1TB [자급제] 최저1,867,590원 https://cr.shopping.naver.com/adcr.nhn?x=ZrSCFFjh0HAPD5LHVggdTP%2F%2F%2Fw%3D%3Ds1qw1PMXJlcmq5qyU1inQAj1QMnT5r38MZlMUBwYG7opN4NgkrPIjlczCBjhk2dSM8qL8zAqB9Ln%2BKBMAr1gGlFULZaC%2Bnx7bchww%2B%2BxpYASjdLOUHr1JEhkMjqfQgywRS0Fv93AoOobzmVCRXa2IsMv%2BjB2erv%2BZ%2Brrs5VHOiTQgGE7%2BZatKjf5hEf1XctkKCqMGcFvs80%2Bm9t1aozn63JbkCnZt3PKDnGtnPfTpWXV9gLRMuJH34gK%2BdKxxm2wL%2BJKrK3wAIdlc1W0GjThbXUPg%2F6PYJW9LpCSumzPJoigK7CtQzZWLlur1%2FFge4%2BEpRw4wFyz5HW6UIpAj3%2BxdImgoIKsc0pNBM1PfaxgJex0bV9SohnsJOp40F4RSK4iNnRZE7VsoXqq4UW%2Bf7klEpS6UxJeah0Om0lkIOZtT9ZLxLbU57LuNzmnFLwGq45KovtNin%2Brw3R1h4tZ%2FeJRj8RQ5Y304PhkZvKPG123kGPO7A%2F0vSPti8cHY4o2fastwY%2Fq%2BhJe%2F9%2FigY7GOLUuI9%2Fn0fhpYscUWfkzfPfQygDURpOFzCbaM6OU97uoPifjrS7nyjInefJHJYX9Lm7Pb3j3UztdMdmsTfHSb%2B2TLiqgdHI8FZtQVGFZJxOyBDKc%2B&nvMid=29029192588&catId=50001519
Apple 아이폰 13 프로 256GB [자급제] 최저2,299,940원 https://cr.shopping.naver.com/adcr.nhn?x=PXB6NX7swf6QEB56PW6oJP%2F%2F%2Fw%3D%3DsTw7RwO%2FlrM%2F5m2TUb6W9ezm1RE9z2b

Apple 아이폰 13 미니 256GB [LG U+] 최저506,490원 https://cr.shopping.naver.com/adcr.nhn?x=JPI73%2BXMWdU1%2B1%2BWLd8pwv%2F%2F%2Fw%3D%3DsHxBk2ofRBscTnjAFYB%2F2lo3mH0KOD4jhJMiKarmtxBSGgjNtjvRyY8TaEm3AQ5Uwmh8s0SrAoXQ5ic7EZkqchEF6FY9ielpYlpF4UkeBpWuxfzS4Ewk4JGVGSCT3oYuDnVtAWrZobZNAYLLYO3j%2Bpt9%2BKy%2Fm0d%2BgYWHtEf61Jo5CNclKwvLyA%2Fys%2F%2FHPPajLeYmWQ3q7Vfmvwq%2BPpoL3AGNmcbgLr0lu2cCLGgyX7c9vsepmwRk8o%2FCB4DgHhnjV3ZSnvvXF0gkneG8ZZ%2Bc8m6sJF1TG8DXvuOXRWh2aiSE1QmeK3ZPITzKHcbEnR%2BI8V3goqQfQ2QONQjaQ8tbuXJpw0XkrPPNQBbpBU3gnLhpcNZN5FF46Jqg7fDWNj5Ca5P%2BxC0UZTSDbeFieBzOyzZkK4pYNbudh0sE02LoO35dDk2nHMP4aHVCx%2FcONoE3ucOT%2BRsdnHIzNMGl8g3WFM5Qad5K03cEQZUE2xf7MbxDyNHyBUXwWVD2ki0TxnNzyDYzfUMBJX%2BZ%2BBkRoR%2Fh87sFBlgE3rGzw6bfD96U%2Be9ERpOFzCbaM6OU97uoPifjrpC6SKJ6ibAU7GpqBYrUq1CfS3ueriC2HHXAuPDPmapKCqA6%2BrrPkSkQjk3bFq%2FUy&nvMid=29030376589&catId=50000248
Apple 아이폰 13 128GB [LG U+] 최저147,500원 https://cr.shopping.naver.com/adcr.nhn?x=WvSJG68nyMLRqnFbsG1COP%2F%2F%2Fw%3D%3DsYuszFzVcr%2F3%2BJHhWTv3gBXkvdDhRexm%2Fn

Apple 아이폰 13 512GB [LG U+] 최저557,000원 https://cr.shopping.naver.com/adcr.nhn?x=hiiPHTYOajF3woCiHqwSH%2F%2F%2F%2Fw%3D%3DsGCwfwUw0KfudbDR2VSNWSxeddPDX%2BffhqEiSGmmvRp%2BGgjNtjvRyY8TaEm3AQ5UweUFsnCmL4DPn4sjaiTIr60F6FY9ielpYlpF4UkeBpWuxfzS4Ewk4JGVGSCT3oYuDnVtAWrZobZNAYLLYO3j%2Bpt9%2BKy%2Fm0d%2BgYWHtEf61Jo5CNclKwvLyA%2Fys%2F%2FHPPajLFE9QF%2F1S7E7CtXP8syJEuWNmcbgLr0lu2cCLGgyX7c9vsepmwRk8o%2FCB4DgHhnjV3ZSnvvXF0gkneG8ZZ%2Bc8m6sJF1TG8DXvuOXRWh2aiSE1QmeK3ZPITzKHcbEnR%2BI83VgtYiJa9IeHf0uTio7C%2F279IfePjtrUAv51%2BesAsN1cNZN5FF46Jqg7fDWNj5Ca1HqFh1ogdBT1KPHTw%2FRddoBD40iqWaNeYMpOus5JTYBDk2nHMP4aHVCx%2FcONoE3uxvd4FH7C8HSLdi0kmjPZ%2FOksp9XuUdeQZ9g2FaJQ4%2B0g1HDVA0X848ldvn7fnwtgcauDZIG8V378qowjUCDLApTL3ruaVJreZl%2BplMuLoOoM1ArTknUek%2FmrLPV4ep%2F4nspQs77yru4T2NhSQqaHA1aFndIwdNwuWmKtx1HNSCrUkiyUGZ3FpPHvE7X48%2Bhb&nvMid=29030296586&catId=50000248
Apple 아이폰 13 프로 128GB [LG U+] 최저1,078,590원 https://cr.shopping.naver.com/adcr.nhn?x=PZXMj%2F2%2B4WBkUYq2q7qZgf%2F%2F%2Fw%3D%3DsI5s8%2F2H2j%2BcbMmDC0Yjw57H6wNvK8

애플 아이폰13 미니 256GB 미개봉 새상품 자급제 통신사 280,000원 https://cr.shopping.naver.com/adcr.nhn?x=PycLPTHn1jI42%2BTZ1nxl2%2F%2F%2F%2Fw%3D%3DswrH%2F3SpvSfU0vaoGsk0AA5tvFH6rm377qtYq9GZlvsiGgjNtjvRyY8TaEm3AQ5UwYeg0NrYWhXMQsONjmEmkKEF6FY9ielpYlpF4UkeBpWuxfzS4Ewk4JGVGSCT3oYuDnVtAWrZobZNAYLLYO3j%2BpkS%2BortKzfQDWBF5GV%2FKfqEhJv3ljJo79VVIpj6%2Fm3WOOzt6Q7AVqzNEn9zz%2FWpEwwl5DBhC6JdGtEAqtKSXLt%2FDpmNu2Rs3tEPrV0ZTODsbWqyaFa2xLbpwGCJChqMDqum%2BQwBMNCuIgmvkVEvKK3Bzmvya0XEt9NKeC9ptScMkDG4evy%2BM13j64UYOnhTyGc4W2cTm1Yjamgw20LigQMR8xFiiHki%2FIAhpKNJriFwEzgKInaGsHN%2BzcOI5%2BwOSJdQtLY1EKF2uzIwLGIY3TL9da5t7S2YYsJBU1Z1eohHKu6BshnnRl5APn8rKTgd9u4cTwhCD7m5MFuO3D78CoXD6B5FUdLl%2Bxe3m8EZE%2Bb1pr1NI9O%2FoiM%2FNnQ4H%2BpKVgnR9INs9ZpxiOekV5%2FkbMXF0f3UUKwrbBDbO%2BREjT5uRiuh6HZniAxwuBCzde3xY4GkIhOFIC9T4X15tHcV2VTyFbDf4gHI5rSMJU1BWcIOg&nvMid=85691062471&catId=50001519
[공식인증점] Apple 아이폰 13 Pro Max+ 애플케어플러스 (선택) 329,000원 https://cr.shopping.naver.com/adcr.nhn?x=n2ve6UD18LkISYj8aFl5wf%2F%2F%2Fw%3D%3Dso70cc29kCm5n2F3tlDllyHInBb5aG

### 수집 후 DF로 저장

In [13]:
name =[]
price = []
link = []

In [14]:
# 전체 상품 정보 추출
items = driver.find_elements(By.CSS_SELECTOR,".basicList_info_area__TWvzp")

In [ ]:
#.price_price__LEGN7

In [15]:
for item in items : 
    name.append(item.find_element(By.CSS_SELECTOR,'.basicList_title__VfX3c').text)
    try :
        price.append(item.find_element(By.CSS_SELECTOR,".price_price__LEGN7").text)
    except : 
        price.append('판매중단')
    link.append(item.find_element(By.CSS_SELECTOR,'.basicList_title__VfX3c > a').get_attribute('href'))


In [16]:
len(name),len(price),len(link)

(46, 46, 46)

In [17]:
pd.DataFrame({'품명':name,'가격':price,'link':link})

,품명,가격,link
0,아이폰 13 mini 자급제 128GB 미드나이트 Apple,"874,000원",https://adcr.naver.com/adcr?x=CxO4zfkTRJPWIb7V...
1,아이폰 13 mini 자급제 128GB 핑크 Apple,"874,000원",https://adcr.naver.com/adcr?x=EGW9TIzWBA4HMhY/...
2,아이폰 13 mini 자급제 256GB 스타라이트 Apple,"997,350원",https://adcr.naver.com/adcr?x=w9uDXNyq7jyAQb4o...
3,Apple 아이폰 13 256GB [자급제],"최저1,168,500원",https://cr.shopping.naver.com/adcr.nhn?x=Y1RY3...
4,Apple 아이폰 13 미니 128GB [자급제],"최저147,870원",https://cr.shopping.naver.com/adcr.nhn?x=lR4%2...
5,Apple 아이폰 13 미니 256GB [자급제],"최저997,350원",https://cr.shopping.naver.com/adcr.nhn?x=KYFCa...
6,Apple 아이폰 13 128GB [자급제],"최저1,057,000원",https://cr.shopping.naver.com/adcr.nhn?x=Ip4na...
7,Apple 아이폰 13 미니 512GB [자급제],"최저1,224,000원",https://cr.shopping.naver.com/adcr.nhn?x=1Gp1v...
8,Apple 아이폰 13 프로 128GB [자급제],"최저1,649,990원",https://cr.shopping.naver.com/adcr.nhn?x=3WQUm...
9,Apple 아이폰 13 프로 512GB [자급제],"최저1,650,000원",https://cr.shopping.naver.com/adcr.nhn?x=MhmTN...
